## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716846939972, experiment_id='0', last_update_time=1716846939972, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import random
mlflow.set_experiment("my-experiment")

2024/05/27 16:55:58 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716846958591, experiment_id='1', last_update_time=1716846958591, lifecycle_stage='active', name='my-experiment', tags={}>

In [10]:
with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": random.choice([0.1, 1, 10]), "random_state": random.randint(0, 100)}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


default artifacts URI: 'mlflow-artifacts:/1/e9aa50f2f2904af280d7260b9a81e5be/artifacts'


/Users/gbemidebe/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/gbemidebe/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716846958591, experiment_id='1', last_update_time=1716846958591, lifecycle_stage='active', name='my-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716846939972, experiment_id='0', last_update_time=1716846939972, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [12]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [13]:
runs = client.search_runs(experiment_ids='1')

for run in runs:
    print(f'id = {run.info.run_id}, accuracy = {run.data.metrics["accuracy"]}')

id = e9aa50f2f2904af280d7260b9a81e5be, accuracy = 0.98
id = dfa9b206cae542a6a3e6b1e47ab03dd7, accuracy = 0.98
id = a7feccade3b94ccaac114883eed1fdc3, accuracy = 0.96
id = 60e256e3c8e54d8087b378dd20cc0e9c, accuracy = 0.96
id = 3e64439b4bba46a1b8105708f0886a65, accuracy = 0.96
id = e946e28a48c94839b7b68db3ba49e242, accuracy = 0.98


In [14]:
model_name = "iris-classifier"

for run in runs:
    mlflow.register_model(
    model_uri=f"runs:/{run.info.run_id}/models",
    name=model_name)

Successfully registered model 'iris-classifier'.
2024/05/27 16:57:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.
Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/05/27 16:57:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.
Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/05/27 16:57:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 3
Created version '3' of model 'iris-classifier'.
Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/05/27 16:57:2

In [15]:
from datetime import datetime

# transistion to staging
model_version = 4
model_name = "iris-classifier"
new_stage = "Staging"
print(f"Transitioning model '{model_name}' version '{model_version}' to stage '{new_stage}'")
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

# annotate the model version with a description
date = datetime.today().date()
client.update_model_version(
    name=model_name, 
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {datetime.today().date()}"
)

Transitioning model 'iris-classifier' version '4' to stage 'Staging'


/var/folders/83/j83q5_mj11956_7k9tnld0ym0000gn/T/ipykernel_6144/4080354918.py:8: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716847047990, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2024-05-27', last_updated_timestamp=1716847449473, name='iris-classifier', run_id='60e256e3c8e54d8087b378dd20cc0e9c', run_link='', source='mlflow-artifacts:/1/60e256e3c8e54d8087b378dd20cc0e9c/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='4'>

In [16]:
# transistion to staging
model_version = 2
print(f"Transitioning model '{model_name}' version '{model_version}' to stage '{new_stage}'")
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

# annotate the model version with a description
date = datetime.today().date()
client.update_model_version(
    name=model_name, 
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {datetime.today().date()}"
)

Transitioning model 'iris-classifier' version '2' to stage 'Staging'


/var/folders/83/j83q5_mj11956_7k9tnld0ym0000gn/T/ipykernel_6144/2801824326.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716847047921, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-05-27', last_updated_timestamp=1716847497993, name='iris-classifier', run_id='dfa9b206cae542a6a3e6b1e47ab03dd7', run_link='', source='mlflow-artifacts:/1/dfa9b206cae542a6a3e6b1e47ab03dd7/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>